# Food Image Classification with ResNet

In [ ]:
import os
import paddle
import paddle.vision.transforms as T
import numpy as np
from PIL import Image
import cv2
from sklearn.utils import shuffle
import paddle.nn.functional as F
import matplotlib.pyplot as plt
from paddle.static import InputSpec
%matplotlib inline


In [ ]:
data_path = 'work/food-11/'  # 设置初始文件地址
character_folders = os.listdir(data_path)  # 查看地址下文件夹
print('character_folders:{}'.format(character_folders))

if(os.path.exists('work/food-11/training.txt')):  # 判断有无文件
    os.remove('work/food-11/training.txt')  # 删除文件
if(os.path.exists('work/food-11/validation.txt')):
    os.remove('work/food-11/validation.txt')
if(os.path.exists('work/food-11/testing.txt')):
    os.remove('work/food-11/testing.txt')

DATA_ROOT = 'work/food-11'

def generate_annotation(mode):
    with open('{}/{}.txt'.format(DATA_ROOT, mode), 'w') as f:
        train_dir = '{}/{}'.format(DATA_ROOT, mode)
        image_path = '{}'.format(train_dir) 
        for image in os.listdir(image_path):
            image_file = '{}/{}'.format(image_path, image)
            label_index = 0 if mode == 'testing' else int(image.split('_')[0])
            try:
                with open(image_file, 'rb') as f_img:
                    img = Image.open(io.BytesIO(f_img.read()))
                    img.load()
                    if img.mode == 'RGB':
                        f.write('{}\t{}\n'.format(image_file, label_index))
            except:
                continue

generate_annotation('training')
generate_annotation('validation')
generate_annotation('testing')

print("Data annotations generated!")


In [ ]:
image_shape = [3, 224, 224]
IMAGE_SIZE = (image_shape[1], image_shape[2])  # [224,224]

class FoodDataset(paddle.io.Dataset):
    def __init__(self, mode='training'):
        assert mode in ['training','validation', 'testing'], 'mode is one of train, valid, test.'
        self.data = []
        with open(f'work/food-11/{mode}.txt') as f:
            for line in f.readlines():
                info = line.strip().split('\t')
                if len(info) > 1:
                    self.data.append([info[0].strip(), info[1].strip()])
        self.transforms = T.Compose([
            T.Resize((256, 256)),
            T.RandomCrop(IMAGE_SIZE) if mode == 'training' else T.CenterCrop(IMAGE_SIZE),
            T.RandomRotation(15) if mode == 'training' else None,
            T.RandomHorizontalFlip(0.5) if mode == 'training' else None,
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    def __getitem__(self, index):
        image_file, label = self.data[index]
        image = Image.open(image_file)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image = self.transforms(image)
        return image, np.array(label, dtype='int64')
    def __len__(self):
        return len(self.data)

training_dataset = FoodDataset(mode='training')
validation_dataset = FoodDataset(mode='validation')
testing_dataset = FoodDataset(mode='testing')

print(f'Training dataset size: {len(training_dataset)}, Validation dataset size: {len(validation_dataset)}, Testing dataset size: {len(testing_dataset)}')


In [ ]:
from paddle.vision.models import resnet50

class ResNetModelPretrained(paddle.nn.Layer):
    def __init__(self, num_classes=11):
        super(ResNetModelPretrained, self).__init__()
        self.base_model = resnet50(pretrained=True)
        self.base_model.fc = paddle.nn.Linear(2048, num_classes)

    def forward(self, x):
        return self.base_model(x)

network = ResNetModelPretrained(num_classes=11)
paddle.summary(network, (-1, 3, 224, 224))


In [ ]:
scheduler = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=0.001, T_max=10)
optim = paddle.optimizer.AdamW(learning_rate=scheduler, parameters=network.parameters())

model = paddle.Model(network)
model.prepare(
    optim,
    paddle.nn.CrossEntropyLoss(),
    paddle.metric.Accuracy()
)

visualdl = paddle.callbacks.VisualDL(log_dir='visualdl_log')
model.fit(
    training_dataset,
    validation_dataset,
    epochs=20,
    batch_size=128,
    verbose=1,
    callbacks=[visualdl]
)

model.save('./finetuning/food-11', training=True)
print("Model saved!")


In [ ]:
network = ResNetModelPretrained(num_classes=11)
model = paddle.Model(network)
model.load('./finetuning/food-11.pdparams')
model.prepare()

result = model.predict(testing_dataset)
LABEL_MAP = [
    "Bread", "Dairy product", "Dessert", "Egg", "Fried food",
    "Meat", "Noodles or pasta", "Rice", "Seafood", "Soup", "Vegetable or fruit"
]

idx = 1520
image_file = testing_dataset.data[idx][0]
predict_label = np.argmax(result[0][idx])
print(f'Sample ID: {idx}, Predicted Label: {predict_label}: {LABEL_MAP[predict_label]}')

image = Image.open(image_file)
plt.figure(figsize=(10, 6))
plt.imshow(image)
plt.title(f'Prediction: {LABEL_MAP[predict_label]}')
plt.show()
